In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x

import tensorflow as tf

import numpy as np
import os
import time



In [0]:
!pip install chembl_webresource_client

In [0]:
from chembl_webresource_client.new_client import new_client

In [0]:
drug_indication = new_client.drug_indication
molecules = new_client.molecule
lung_cancer_ind = drug_indication.filter(efo_term__icontains="LUNG CARCINOMA")
lung_cancer_mols = molecules.filter(molecule_chembl_id__in=[x['molecule_chembl_id'] for x in lung_cancer_ind])

In [0]:
lung_cancer_mols

In [0]:
stringCol = ""

for i in range (0,128):
  if lung_cancer_mols[i]["molecule_structures"] != None:
    if lung_cancer_mols[i]["molecule_structures"]['canonical_smiles'] != None:
      stringCol = stringCol + " " + lung_cancer_mols[i]["molecule_structures"]['canonical_smiles']
    
f = open("smiles.txt", "w+")    
f.write(stringCol)
f.close()

In [0]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
import tensorflow as tf

In [0]:
#open smile file
filename = "smiles.txt"
raw_text = open(filename, 'r', encoding='utf-8').read()


vocab = sorted(list(set(raw_text)))
char_to_idx = {u:i for i, u in enumerate(vocab)}
idx_to_char = np.array(vocab)

mol_as_int = np.array([char_to_idx[c] for c in raw_text])

mol_dataset = tf.data.Dataset.from_tensor_slices(mol_as_int)

# summarize the loaded data
n_chars = len(raw_text)
n_vocab = len(idx_to_char)


seq_length = 25
examples_per_epoch = len(raw_text)//(seq_length+1)
sequences = mol_dataset.batch(seq_length+1, drop_remainder=True)


In [0]:
#create dataset

def create_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(create_input_target)

In [0]:
#batching


BATCH_SIZE = 64


BUFFER_SIZE = 1000
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
dataset

In [0]:
#model

def build_model(n_vocab, dmembed, lstm_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(n_vocab, dmembed,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.LSTM(lstm_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(n_vocab,activation = 'softmax')
    
  ])
  return model

In [0]:
n_vocab

In [0]:
#create model

dmembed = 256

lstm_units = 200

model = build_model(
  n_vocab = n_vocab,
  dmembed=dmembed,
  lstm_units=lstm_units,
  batch_size=BATCH_SIZE)


In [0]:
#loss calc
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

model.compile(optimizer='adam', loss=loss)

In [0]:
import os

# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)


In [0]:
version = model.fit(dataset, epochs=20, callbacks=[checkpoint_callback])

In [0]:
tf.train.latest_checkpoint(checkpoint_dir)

model = build_model(n_vocab, dmembed, lstm_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [0]:




def generate_smiles(model, start_string):
 
  mols_generate = 1000


  input_data = [char_to_idx[s] for s in start_string]
  input_data = tf.expand_dims(input_eval, 0)


  smiles_generated = []

  
  variation = 2.0

 
  model.reset_states()
  for i in range(mols_generate):
      predictions = model(input_data)

      predictions = tf.squeeze(predictions, 0)

     
      predictions = predictions / variation
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

     
      input_eval = tf.expand_dims([predicted_id], 0)

      smiles_generated.append(idx_to_char[predicted_id])

  return (start_string + ''.join(smiles_generated))

In [0]:
seed = #CN1CCC[C@H]1c2cccnc
gen_smiles = generate_smiles(model, start_string=u"")
print(gen_smiles)

In [0]:
import sys
import os
import requests
import subprocess
import shutil
from logging import getLogger, StreamHandler, INFO


logger = getLogger(__name__)
logger.addHandler(StreamHandler())
logger.setLevel(INFO)


def install(
        chunk_size=4096,
        file_name="Miniconda3-latest-Linux-x86_64.sh",
        url_base="https://repo.continuum.io/miniconda/",
        conda_path=os.path.expanduser(os.path.join("~", "miniconda")),
        rdkit_version=None,
        add_python_path=True,
        force=False):
    """install rdkit from miniconda
    ```
    import rdkit_installer
    rdkit_installer.install()
    ```
    """

    python_path = os.path.join(
        conda_path,
        "lib",
        "python{0}.{1}".format(*sys.version_info),
        "site-packages",
    )

    if add_python_path and python_path not in sys.path:
        logger.info("add {} to PYTHONPATH".format(python_path))
        sys.path.append(python_path)

    if os.path.isdir(os.path.join(python_path, "rdkit")):
        logger.info("rdkit is already installed")
        if not force:
            return

        logger.info("force re-install")

    url = url_base + file_name
    python_version = "{0}.{1}.{2}".format(*sys.version_info)

    logger.info("python version: {}".format(python_version))

    if os.path.isdir(conda_path):
        logger.warning("remove current miniconda")
        shutil.rmtree(conda_path)
    elif os.path.isfile(conda_path):
        logger.warning("remove {}".format(conda_path))
        os.remove(conda_path)

    logger.info('fetching installer from {}'.format(url))
    res = requests.get(url, stream=True)
    res.raise_for_status()
    with open(file_name, 'wb') as f:
        for chunk in res.iter_content(chunk_size):
            f.write(chunk)
    logger.info('done')

    logger.info('installing miniconda to {}'.format(conda_path))
    subprocess.check_call(["bash", file_name, "-b", "-p", conda_path])
    logger.info('done')

    logger.info("installing rdkit")
    subprocess.check_call([
        os.path.join(conda_path, "bin", "conda"),
        "install",
        "--yes",
        "-c", "rdkit",
        "python=={}".format(python_version),
        "rdkit" if rdkit_version is None else "rdkit=={}".format(rdkit_version)])
    logger.info("done")

    import rdkit
    logger.info("rdkit-{} installation finished!".format(rdkit.__version__))


if __name__ == "__main__":
    install()

In [0]:
from rdkit import Chem

genMols = gen_smiles.split()
valid_sanitized_mols = []

for mol in genMols:
  m = Chem.MolFromSmiles(mol,sanitize=False)
  if m is None:
    print('invalid SMILES')
  else:
    try:
      Chem.SanitizeMol(m)
    except:
      print('invalid chemistry')
  valid_sanitized_mols.append(Chem.MolToSmiles(m))
    

In [0]:
outputfile = open("valid_smiles.txt", "w+")
f.write(valid_sanitized_mols)
f.close()